In [2]:
from vivarium import Artifact
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from db_queries import get_ids, get_outputs
import scipy.stats
import db_queries

!whoami
!date

alibow
Wed Jun 24 10:35:59 PDT 2020


In [5]:
output_dirs = ['/share/costeffectiveness/results/vivarium_conic_lsff/00_bugfix/india_new_start_date/count_data/']

locations = ['India']

In [6]:
#NOTE: this was an incomplete run, so comparisons across scenarios are not valid

In [7]:
births = pd.read_hdf(output_dirs[0] + 'births.hdf')
births.head()

,year,sex,folic_acid_fortification_group,measure,input_draw,scenario,value
0,2020,female,covered,live_births,357,baseline,89.0
1,2020,female,covered,live_births,357,baseline,1.0
2,2020,female,covered,live_births,357,baseline,27.0
3,2020,female,covered,live_births,357,iron_fortification_scale_up,89.0
4,2020,female,covered,live_births,357,iron_fortification_scale_up,1.0


In [8]:
births_by_scenario = births.groupby(['input_draw','scenario','sex']).sum().groupby(['scenario','sex']).mean()
births_by_scenario

value
scenario                         sex            
baseline                         female   9079.0
                                 male    10034.0
iron_fortification_scale_up      female   9079.0
                                 male    10034.0
vitamin_a_fortification_scale_up female   9079.0
                                 male    10034.0

In [10]:
ntd_births_by_scenario = ntd_births.groupby(['input_draw','scenario','sex']).sum().groupby(['scenario','sex']).mean()
ntd_births_by_scenario

NameError: name 'ntd_births' is not defined

In [ ]:
ntd_bp = ntd_births_by_scenario / births_by_scenario
ntd_bp

# Check artifact NTD BP

In [8]:
art = Artifact('/share/costeffectiveness/artifacts/vivarium_conic_lsff/india.hdf', 
                   filter_terms=['year_start == 2017', 'age_start < 5'])
data = art.load('cause.neural_tube_defects.birth_prevalence')
data_mean = pd.DataFrame(data.mean(axis=1)).rename(columns={0:'value'})
data_limits = pd.DataFrame(data.quantile([0.025,0.975], axis=1)).transpose().rename(columns={0.025:'lower',
                                                                                             0.975:'upper'})
data = data_mean.merge(data_limits, right_index=True, left_index=True).reset_index()
data.head()

,location,sex,year_start,year_end,value,lower,upper
0,India,Female,2017,2018,0.000762,0.000678,0.000852
1,India,Male,2017,2018,0.000714,0.000636,0.000801
